## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

# Решение

### 1. Обработать данные, получить токены, обработать токены.

In [1]:
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

data = []
morph = pymorphy2.MorphAnalyzer()

# Read Text
with open("data/news_train.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        vec = []
        category, topic, content = line.split("\t")
        vec.append(category)
        vec.append(topic)
        vec.append(content)
        data.append(vec)

# Tokenize

print("Get tokens")

data_tokens = []

for vec in data:
    vec_tokenize = [vec[0]]
    vec_tokenize.append(word_tokenize(vec[1]))
    vec_tokenize.append(word_tokenize(vec[2]))
    data_tokens.append(vec_tokenize)

# Process tokens

counter = 0

print("Analyse Tokens")

for vec in data_tokens:
    for tokens in vec:
        for token in tokens:
            print(morph.parse(token))
            counter += 1
            if counter > 10:
                break
        if counter > 10:
            break
    if counter > 10:
        break


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crewd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Get tokens
Analyse Tokens
[Parse(word='s', tag=OpencorporaTag('LATN'), normal_form='s', score=1.0, methods_stack=((LatinAnalyzer(score=0.9), 's'),))]
[Parse(word='p', tag=OpencorporaTag('LATN'), normal_form='p', score=1.0, methods_stack=((LatinAnalyzer(score=0.9), 'p'),))]
[Parse(word='o', tag=OpencorporaTag('LATN'), normal_form='o', score=1.0, methods_stack=((LatinAnalyzer(score=0.9), 'o'),))]
[Parse(word='r', tag=OpencorporaTag('LATN'), normal_form='r', score=1.0, methods_stack=((LatinAnalyzer(score=0.9), 'r'),))]
[Parse(word='t', tag=OpencorporaTag('LATN'), normal_form='t', score=1.0, methods_stack=((LatinAnalyzer(score=0.9), 't'),))]
[Parse(word='овечкин', tag=OpencorporaTag('NOUN,anim,masc,Sgtm,Surn sing,nomn'), normal_form='овечкин', score=1.0, methods_stack=((DictionaryAnalyzer(), 'овечкин', 37, 0),))]
[Parse(word='пожертвовал', tag=OpencorporaTag('VERB,perf,tran masc,sing,past,indc'), normal_form='пожертвовать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'пожертвовал', 74

### 2. Применить word2vec